# **-----GoodReads Scraped Data Preprocessing Notebook-----**

Authors: Andrew Wieber<br>
Date: January 20th 2024<br>
Python version: 3.11.5

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os
from datetime import datetime, date, time
from scipy import stats

## **SUMMARY**

1) Import the main project data (exported from the main project just after correcting the importation errors)
2) Import the scraped dataframe (using ISBN10 on GoodReads)
3) Import the scraped dataframe (using ISBN13 on GoodReads)
4) Join the ISBN10 and ISBN13 and compare the data: what is different and what is missing?
5) Remove extraneous information from the format column and group similar items
6) Export the data to a file

### **1) IMPORT THE MAIN PROJECT DATA**

In [2]:
#Enter the repertory for all project files:
repPath = "C:/Users/Andrew/OneDrive - Data ScienceTech Institute/Documents/DSTI/4) Machine Learning with Python Labs/Project/Project 1 - CHOSEN/Scraped_final/"

#Import the data file
dfGoodReads = pd.read_csv(repPath + "GoodReads_df_exported_for_post-scraping_treatment.tsv", sep = '\t' , dtype = {'isbn': str, 'isbn13': str}, index_col=0, on_bad_lines = 'warn')

pd.set_option('display.max_columns', None)

#Show the data:
dfGoodReads.head(3)

bookID                                              title  \
0       1  Harry Potter and the Half-Blood Prince (Harry ...   
1       2  Harry Potter and the Order of the Phoenix (Har...   
2       4  Harry Potter and the Chamber of Secrets (Harry...   

                      authors  average_rating        isbn         isbn13  \
0  J.K. Rowling/Mary GrandPré            4.57  0439785960  9780439785969   
1  J.K. Rowling/Mary GrandPré            4.49  0439358078  9780439358071   
2                J.K. Rowling            4.42  0439554896  9780439554893   

  language_code  num_pages  ratings_count  text_reviews_count  \
0           eng        652        2095690               27591   
1           eng        870        2153167               29221   
2           eng        352           6333                 244   

  publication_date        publisher  
0        9/16/2006  Scholastic Inc.  
1         9/1/2004  Scholastic Inc.  
2        11/1/2003       Scholastic

### **2) IMPORT DATA SCRAPED USING ISBN10**

In [3]:
#Import the scraped data and add bookID
fileNameScrapedData= "GoodReads_scraped_data_FULL_on_ISBN10.tsv"

data_types = {'isbn13': str, 'isbn': str, 'format': str, 'genre': str}
dfGR_Scraped_on_ISBN10 = pd.read_csv(repPath + fileNameScrapedData, dtype = data_types, sep = '\t', index_col = 0)

#Join the bookID onto the dataframe by merging with the main dataframe
dfGR_Scraped_on_ISBN10 = dfGR_Scraped_on_ISBN10.merge(dfGoodReads[['bookID','isbn']], on='isbn', how='inner')
dfGR_Scraped_on_ISBN10.head(3)

isbn13        isbn                            format  \
0  9780439785969  0439785960              652 pages, Paperback   
1  9780439655484  043965548X  435 pages, Mass Market Paperback   
2  9780439358071  0439358078              870 pages, Paperback   

                                               genre  bookID  
0  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       1  
1  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...       5  
2  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       2

In [4]:
#Check for NaN cells
dfGR_Scraped_on_ISBN10.isna().sum()

isbn13     0
isbn       0
format    40
genre      0
bookID     0
dtype: int64

### **3) IMPORT DATA SCRAPED USING ISBN13**

In [5]:
#Import the scraped data and add bookID
fileNameScrapedData= "GoodReads_scraped_data_FULL_on_ISBN13.tsv"

data_types = {'isbn13': str, 'isbn': str, 'format': str, 'genre': str}
dfGR_Scraped_on_ISBN13 = pd.read_csv(repPath + fileNameScrapedData, dtype = data_types, sep = '\t', index_col = 0)

#Join the bookID onto the dataframe by merging with the main dataframe
dfGR_Scraped_on_ISBN13 = dfGR_Scraped_on_ISBN13.merge(dfGoodReads[['bookID','isbn']], on='isbn', how='inner')
dfGR_Scraped_on_ISBN13.head(3)

isbn13        isbn                            format  \
0  9780439655484  043965548X  435 pages, Mass Market Paperback   
1  9780439358071  0439358078              870 pages, Paperback   
2  9780439785969  0439785960              652 pages, Paperback   

                                               genre  bookID  
0  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...       5  
1  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       2  
2  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       1

In [6]:
#Check for NaN cells, see that there are some formats missing.
#There are also missing genres, but there is a double bracket in this case, so it does not show up here.
dfGR_Scraped_on_ISBN13.isna().sum()

isbn13     0
isbn       0
format    42
genre      0
bookID     0
dtype: int64

### **4) JOIN THE ISBN10 AND ISBN13 DATAFRAMES AND FIND THE DIFFERENCES AND MISSING DATA**

**4a) Create the joined dataframes**

In [7]:
#Join on isbn10 with dfGR_Scraped_on_ISBN10 since it has all the lines (dfGR_Scraped_on_ISBN13 is missing a few lines)
dfGR_Scraped_union_ISBN10_ISB13 = dfGR_Scraped_on_ISBN10.merge(dfGR_Scraped_on_ISBN13, on='isbn', how='inner')
dfGR_Scraped_union_ISBN10_ISB13.head(3)

isbn13_x        isbn                          format_x  \
0  9780439785969  0439785960              652 pages, Paperback   
1  9780439655484  043965548X  435 pages, Mass Market Paperback   
2  9780439358071  0439358078              870 pages, Paperback   

                                             genre_x  bookID_x       isbn13_y  \
0  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...         1  9780439785969   
1  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...         5  9780439655484   
2  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...         2  9780439358071   

                           format_y  \
0              652 pages, Paperback   
1  435 pages, Mass Market Paperback   
2              870 pages, Paperback   

                                             genre_y  bookID_y  
0  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...         1  
1  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...         5  
2  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...         2

In [8]:
#Check that ISBN13s are the same when we join on ISBN10
different_ISB13 = []
listdf = dfGR_Scraped_union_ISBN10_ISB13[['isbn13_x', 'isbn13_y']].values.tolist()
different_ISB13 = [[isbn13_x, isbn13_y] for isbn13_x, isbn13_y in listdf if isbn13_x!=isbn13_y]
print("List of items that are different between the dataframes:", different_ISB13)

List of items that are different between the dataframes: []


**4b) Find the lines where the format is different between the two scrapings (excluding NaNs)**

In [9]:
different_formats = []
listdf = dfGR_Scraped_union_ISBN10_ISB13[['isbn', 'format_x', 'isbn13_y', 'format_y']].values.tolist()
different_formats = [[isbn10, format10, isbn13, format13] for isbn10, format10, isbn13, format13 in listdf if format10!=format13 and not (pd.isna(format10) or pd.isna(format13))]
different_formats = pd.DataFrame(different_formats, columns = ['isbn10', 'format_isbn10', 'isbn13_y', 'format_isbn13'])
different_formats = different_formats.merge(dfGoodReads[['bookID','title', 'authors', 'num_pages','isbn']], left_on='isbn10', right_on='isbn', how='inner')
different_formats

isbn10          format_isbn10       isbn13_y  \
0  0156659751   272 pages, Paperback  9780156659758   
1  0061139637    96 pages, Paperback  9780061139635   
2  0060790598   320 pages, Paperback  9780060790592   
3  0006280560   160 pages, Paperback  9790007672386   
4  0451527747   239 pages, Paperback  9780451527745   
5  0553585975  1264 pages, Paperback  9780553585971   
6  9861146091   183 pages, Paperback  9789861146096   
7  1402714548   197 pages, Hardcover  9781402714542   
8  0060014318   352 pages, Paperback  9780060014315   

                format_isbn13  bookID  \
0        254 pages, Paperback    9813   
1         98 pages, Paperback   12055   
2        316 pages, Paperback   16697   
3                   Paperback   17267   
4        240 pages, Paperback   24213   
5        803 pages, Paperback   25698   
6  183 pages, Unknown Binding   31992   
7        208 pages, Hardcover   38643   
8        340 pages, Paperback   43273   

                                               title  \
0       The Nonexistent Knight & The Cloven Viscount   
1                                     Points in Time   
2                        The Mysteries of Pittsburgh   
3                                  The Great Divorce   
4  Alice's Adventures in Wonderland & Through the...   
5  An Inquiry into the Nature and Causes of the W...   
6                                           鋼之鍊金術師 1   
7                                  A Little Princess   
8  Kids Are Worth It!: Giving Your Child the Gift...   

                                     authors  num_pages        isbn  
0                              Italo Calvino        272  0156659751  
1                                Paul Bowles         96  0061139637  
2                             Michael Chabon        320  0060790598  
3                                 C.S. Lewis        160  0006280560  
4  Lewis Carroll/John Tenniel/Martin Gardner        239  0451527747  
5                                 Adam Smith       1264  0553585975  
6                     Hiromu Arakawa/荒川弘/方郁仁        183  9861146091  
7      Frances Hodgson Burnett/Scott McKowen        208  1402714548  
8                           Barbara Coloroso        352  0060014318

In [10]:
#Create a list of bookIDs for which the isbn13 scraping should be used instead of the isbn10 scraping
bookIDs_from_ISBN13_scraping = [38643]

**4c) Find the lines where the isbn10 scraping gives NaN but the isbn13 scraping gives results**

In [11]:
ISBN13_formats_only_found = []
listdf = dfGR_Scraped_union_ISBN10_ISB13[['isbn', 'format_x', 'isbn13_y', 'format_y', 'genre_x', 'genre_y']].values.tolist()
ISBN13_formats_only_found = [[isbn10, format10, isbn13, format13, genre10, genre13] for isbn10, format10, isbn13, format13, genre10, genre13 in listdf if format10!=format13 and pd.isna(format10) and not pd.isna(format13)]
ISBN13_formats_only_found = pd.DataFrame(ISBN13_formats_only_found, columns = ['isbn10', 'format_isbn10', 'isbn13_y', 'format_isbn13', 'genre_isbn10', 'genre_isbn13'])
ISBN13_formats_only_found = ISBN13_formats_only_found.merge(dfGoodReads[['bookID', 'title', 'authors', 'num_pages','isbn']], left_on='isbn10', right_on='isbn', how='inner')
ISBN13_formats_only_found

isbn10  format_isbn10       isbn13_y  \
0   0312349486            NaN  9780312349486   
1   0060792175            NaN  9780060792176   
2   0375413316            NaN  9780375413315   
3   1400033411            NaN  9781400033416   
4    084386874            NaN  9780842386876   
5   0143039563            NaN  9780143039563   
6   1932664084            NaN  9781932664089   
7   0870541854            NaN  9780870541858   
8   1568820402            NaN  9781568820408   
9   0801882982            NaN  9780801882982   
10  0312257376            NaN  9780312257378   
11  0451529944            NaN  9780451529947   
12  074348777X            NaN  9780743487771   
13  9781903254            NaN  9781903254387   
14  4490249512            NaN  9780449015414   
15  1564775003            NaN  9781564775009   

                       format_isbn13 genre_isbn10  \
0               310 pages, Hardcover           []   
1               276 pages, Paperback           []   
2              1313 pages, Hardcover           []   
3               324 pages, Paperback           []   
4               369 pages, Paperback           []   
5               244 pages, Paperback           []   
6               168 pages, Paperback           []   
7               237 pages, Hardcover           []   
8               260 pages, Paperback           []   
9               320 pages, Paperback           []   
10              188 pages, Paperback           []   
11              144 pages, Paperback           []   
12  208 pages, Mass Market Paperback           []   
13              544 pages, Hardcover           []   
14              192 pages, Paperback           []   
15               80 pages, Paperback           []   

                                         genre_isbn13  bookID  \
0   ['Mystery', 'Fiction', 'Humor', 'Chick Lit', '...    3507   
1   ['Fiction', 'Historical Fiction', 'Contemporar...    4583   
2   ['Fiction', 'Historical Fiction', 'Classics', ...    5488   
3   ['Fiction', 'Classics', 'Historical Fiction', ...    6149   
4   ['Christian Fiction', 'Christian', 'Fiction', ...   11436   
5   ['Classics', 'Fiction', 'Adventure', 'Historic...   24583   
6   ['Graphic Novels', 'Comics', 'Fiction', 'Young...   29800   
7                                          ['Horror']   36505   
8   ['Horror', 'Cthulhu Mythos', 'Lovecraftian', '...   36507   
9                                       ['Economics']   36522   
10  ['Nonfiction', 'History', 'Classics', 'Biograp...   36530   
11  ['Nonfiction', 'History', 'Classics', 'Biograp...   36532   
12  ['Nonfiction', 'History', 'Classics', 'Biograp...   36533   
13  ['Nonfiction', 'Music', 'Horror', 'Occult', 'R...   37063   
14  ['Science Fiction', 'Fiction', 'Science Fictio...   41824   
15  ['Quilting', 'Sewing', 'Nonfiction', 'Crafts',...   43282   

                                                title  \
0                  Twelve Sharp (Stephanie Plum  #12)   
1                           Everything Is Illuminated   
2   The Cairo Trilogy: Palace Walk / Palace of Des...   
3                                             Beloved   
4                            Rejoice (Redemption  #4)   
5   The Adventures of Tom Sawyer (Adventures of To...   
6   Scott Pilgrim  Volume 1: Scott Pilgrim's Preci...   
7                                            Evermore   
8    The Azathoth Cycle: Tales of the Blind Idiot God   
9   Rethinking the Economics of War: The Intersect...   
10  Narrative of the Life of Frederick Douglass: A...   
11  Narrative of the Life of Frederick Douglass: A...   
12  Narrative of the Life of Frederick Douglass: A...   
13  The Gospel of Filth: A Bible of Decadence & Da...   
14        The Currents of Space (Galactic Empire  #2)   
15  Happy Endings: Finishing the Edges of Your Quilts   

                                              authors  num_pages        isbn  
0                                     Janet Evanovich        310  0312349486  
1                                Jonathan Safran Foer  

In [12]:
#Update the list of bookIDs for which the isbn13 scraping should be used
bookIDs_from_ISBN13_scraping = bookIDs_from_ISBN13_scraping + list(ISBN13_formats_only_found.bookID)
print(bookIDs_from_ISBN13_scraping)

[38643, 3507, 4583, 5488, 6149, 11436, 24583, 29800, 36505, 36507, 36522, 36530, 36532, 36533, 37063, 41824, 43282]


**4d) Make a common scraped dataframe using the isbn10 scraping and correcting the errors**

In [13]:
#Create a primary scraped database starting from the ISBN10 file, which contains all entries (ISBN13 is missing some lines)
dfGR_Scraped_Primary = dfGR_Scraped_on_ISBN10
dfGR_Scraped_Primary.head(3)

isbn13        isbn                            format  \
0  9780439785969  0439785960              652 pages, Paperback   
1  9780439655484  043965548X  435 pages, Mass Market Paperback   
2  9780439358071  0439358078              870 pages, Paperback   

                                               genre  bookID  
0  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       1  
1  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...       5  
2  ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       2

In [14]:
#Replace with the bad isbn10 data with the isbn13 replacements
for id_of_book in bookIDs_from_ISBN13_scraping:
    dfGR_Scraped_Primary.loc[dfGR_Scraped_Primary.bookID == id_of_book, "format"] = dfGR_Scraped_on_ISBN13.loc[dfGR_Scraped_on_ISBN13.bookID == id_of_book, "format"].item()
    dfGR_Scraped_Primary.loc[dfGR_Scraped_Primary.bookID == id_of_book, "genre"] = dfGR_Scraped_on_ISBN13.loc[dfGR_Scraped_on_ISBN13.bookID == id_of_book, "genre"].item()
dfGR_Scraped_Primary[dfGR_Scraped_Primary.bookID.isin(bookIDs_from_ISBN13_scraping)]

isbn13        isbn                            format  \
1029   9780312349486  0312349486              310 pages, Hardcover   
1297   9780060792176  0060792175              276 pages, Paperback   
1589   9780375413315  0375413316             1313 pages, Hardcover   
1738   9781400033416  1400033411              324 pages, Paperback   
3107   9780842386876   084386874              369 pages, Paperback   
6507   9780143039563  0143039563              244 pages, Paperback   
7720   9781932664089  1932664084              168 pages, Paperback   
9281   9780870541858  0870541854              237 pages, Hardcover   
9282   9781568820408  1568820402              260 pages, Paperback   
9283   9780801882982  0801882982              320 pages, Paperback   
9284   9780312257378  0312257376              188 pages, Paperback   
9285   9780451529947  0451529944              144 pages, Paperback   
9286   9780743487771  074348777X  208 pages, Mass Market Paperback   
9350   9781903254387  9781903254              544 pages, Hardcover   
9674   9781402714542  1402714548              208 pages, Hardcover   
10320  9780449015414  4490249512              192 pages, Paperback   
10598  9781564775009  1564775003               80 pages, Paperback   

                                                   genre  bookID  
1029   ['Mystery', 'Fiction', 'Humor', 'Chick Lit', '...    3507  
1297   ['Fiction', 'Historical Fiction', 'Contemporar...    4583  
1589   ['Fiction', 'Historical Fiction', 'Classics', ...    5488  
1738   ['Fiction', 'Classics', 'Historical Fiction', ...    6149  
3107   ['Christian Fiction', 'Christian', 'Fiction', ...   11436  
6507   ['Classics', 'Fiction', 'Adventure', 'Historic...   24583  
7720   ['Graphic Novels', 'Comics', 'Fiction', 'Young...   29800  
9281                                          ['Horror']   36505  
9282   ['Horror', 'Cthulhu Mythos', 'Lovecraftian', '...   36507  
9283                                       ['Economics']   36522  
9284   ['Nonfiction', 'History', 'Classics', 'Biograp...   36530  
9285   ['Nonfiction', 'History', 'Classics', 'Biograp...   36532  
9286   ['Nonfiction', 'History', 'Classics', 'Biograp...   36533  
9350   ['Nonfiction', 'Music', 'Horror', 'Occult', 'R...   37063  
9674   ['Classics', 'Fiction', 'Childrens', 'Young Ad...   38643  
10320  ['Science Fiction', 'Fiction', 'Science Fictio...   41824  
10598  ['Quilting', 'Sewing', 'Nonfiction', 'Crafts',...   43282

**4e) Get the missing format information**

In [15]:
#Output the missing formats
missing_formats = []
listdf = dfGR_Scraped_union_ISBN10_ISB13[['isbn', 'format_x', 'isbn13_y', 'format_y']].values.tolist()
missing_formats = [[isbn10, format10, isbn13, format13] for isbn10, format10, isbn13, format13 in listdf if format10!=format13 and (pd.isna(format10) and pd.isna(format13))]
missing_formats = pd.DataFrame(missing_formats, columns = ['isbn10', 'format_isbn10', 'isbn13_y', 'format_isbn13'])
missing_formats

isbn10  format_isbn10       isbn13_y  format_isbn13
0   2742741461            NaN  9782742741465            NaN
1   0743564677            NaN  9780743564670            NaN
2   074353395X            NaN  9780743533959            NaN
3   1595191356            NaN  9781595191359            NaN
4   0684873176            NaN  9780684873176            NaN
5   0307237583            NaN  9780739474792            NaN
6   0613647904            NaN  9780613647908            NaN
7   0785759379            NaN  9780785759379            NaN
8   5040019521            NaN  9785040019526            NaN
9   0822003902            NaN  9780822003908            NaN
10  0563388692            NaN  9780563388692            NaN
11  0563388323            NaN  9780563388326            NaN
12  0613046889            NaN  9780613046886            NaN
13  0613589459            NaN  9780613589451            NaN
14  0613674839            NaN  9780613674836            NaN
15  061358466X            NaN  9780613584661            NaN
16  0679451218            NaN  9780679451211            NaN
17  0439959047            NaN  9780439959049            NaN
18  0881031909            NaN  9780881031904            NaN
19  8401474728            NaN  9788401474729            NaN
20  0613572173            NaN  9780613572170            NaN
21  1932051295            NaN  9781932051292            NaN
22  0756615690            NaN  9780756615697            NaN
23  1416500502            NaN  9781416500506            NaN

***>>>A file is manually created and filled with this ISBN10 and ISBN13 data. It is called "Missing_info_by_ISBNs_Manually_Entered.tsv"***

In [16]:
#Import the missing data coming from the manually created file
fileNameScrappedData= "Missing_info_by_ISBNs_Manually_Entered.tsv"

data_types = {'isbn13': str, 'isbn': str, 'format': str, 'genre': str}

dfGR_Scraped_Missing_Info = pd.read_csv(repPath + fileNameScrappedData, dtype = data_types, sep = '\t', index_col = 0)
dfGR_Scraped_Missing_Info = dfGR_Scraped_Missing_Info.merge(dfGoodReads[['bookID','isbn']], left_on='isbn10', right_on='isbn', how='inner')
dfGR_Scraped_Missing_Info

isbn13      isbn10           format genre  bookID        isbn
0   9782742741465  2742741461   Product Bundle    []     466  2742741461
1   9780743564670  0743564677         Audio CD    []    3593  0743564677
2   9780743533959  074353395X         Audio CD    []    3599  074353395X
3   9781595191359  1595191356   Product Bundle    []    8077  1595191356
4   9780684873176  0684873176        Hardcover    []   10876  0684873176
5   9780739474792  0307237583        Hardcover    []   13121  0307237583
6   9780613647908  0613647904  Library Binding    []   15900  0613647904
7   9780785759379  0785759379  Library Binding    []   17511  0785759379
8   9785040019526  5040019521        Paperback    []   17837  5040019521
9   9780822003908  0822003902        Paperback    []   18984  0822003902
10  9780563388692  0563388692         Audio CD    []   19786  0563388692
11  9780563388326  0563388323         Audio CD    []   19788  0563388323
12  9780613046886  0613046889  Library Binding    []   23479  0613046889
13  9780613589451  0613589459  Library Binding    []   24060  0613589459
14  9780613674836  0613674839  Library Binding    []   24062  0613674839
15  9780613584661  061358466X  Library Binding    []   24066  061358466X
16  9780679451211  0679451218       Flexibound    []   26805  0679451218
17  9780439959049  0439959047        Paperback    []   27877  0439959047
18  9780881031904  0881031909  Library Binding    []   28396  0881031909
19  9788401474729  8401474728        Paperback    []   32679  8401474728
20  9780613572170  0613572173  Library Binding    []   34530  0613572173
21  9781932051292  1932051295        Paperback    []   36461  1932051295
22  9780756615697  0756615690       Flexibound    []   38238  0756615690
23  9781416500506  1416500502        Hardcover    []   43491  1416500502

**4f) Replace the format and genre information in the lines with the missing data**

In [17]:
#Get the list of bookIDs for which the "missing data" file should be used
bookIDs_from_missing_data_file = list(dfGR_Scraped_Missing_Info.bookID)
print(bookIDs_from_missing_data_file)

[466, 3593, 3599, 8077, 10876, 13121, 15900, 17511, 17837, 18984, 19786, 19788, 23479, 24060, 24062, 24066, 26805, 27877, 28396, 32679, 34530, 36461, 38238, 43491]


In [18]:
#Check that the formats are all NaN
dfGR_Scraped_Primary[dfGR_Scraped_Primary.bookID.isin(bookIDs_from_missing_data_file)]

isbn13        isbn format genre  bookID
199    9782742741465  2742741461    NaN    []     466
1059   9780743564670  0743564677    NaN    []    3593
1061   9780743533959  074353395X    NaN    []    3599
2252   9781595191359  1595191356    NaN    []    8077
2928   9780684873176  0684873176    NaN    []   10876
3618   9780739474792  0307237583    NaN    []   13121
4416   9780613647908  0613647904    NaN    []   15900
4851   9780785759379  0785759379    NaN    []   17511
4930   9785040019526  5040019521    NaN    []   17837
5256   9780822003908  0822003902    NaN    []   18984
5470   9780563388692  0563388692    NaN    []   19786
5472   9780563388326  0563388323    NaN    []   19788
6220   9780613046886  0613046889    NaN    []   23479
6387   9780613589451  0613589459    NaN    []   24060
6388   9780613674836  0613674839    NaN    []   24062
6392   9780613584661  061358466X    NaN    []   24066
7038   9780679451211  0679451218    NaN    []   26805
7256   9780439959049  0439959047    NaN    []   27877
7367   9780881031904  0881031909    NaN    []   28396
8502   9788401474729  8401474728    NaN    []   32679
8916   9780613572170  0613572173    NaN    []   34530
9277   9781932051292  1932051295    NaN    []   36461
9586   9780756615697  0756615690    NaN    []   38238
10665  9781416500506  1416500502    NaN    []   43491

In [19]:
#Replace format and genre for these bookIDs
for id_of_book in bookIDs_from_missing_data_file:
    dfGR_Scraped_Primary.loc[dfGR_Scraped_Primary.bookID == id_of_book, "format"] = dfGR_Scraped_Missing_Info.loc[dfGR_Scraped_Missing_Info.bookID == id_of_book, "format"].item()
    dfGR_Scraped_Primary.loc[dfGR_Scraped_Primary.bookID == id_of_book, "genre"] = dfGR_Scraped_Missing_Info.loc[dfGR_Scraped_Missing_Info.bookID == id_of_book, "genre"].item()
dfGR_Scraped_Primary[dfGR_Scraped_Primary.bookID.isin(bookIDs_from_missing_data_file)]

isbn13        isbn           format genre  bookID
199    9782742741465  2742741461   Product Bundle    []     466
1059   9780743564670  0743564677         Audio CD    []    3593
1061   9780743533959  074353395X         Audio CD    []    3599
2252   9781595191359  1595191356   Product Bundle    []    8077
2928   9780684873176  0684873176        Hardcover    []   10876
3618   9780739474792  0307237583        Hardcover    []   13121
4416   9780613647908  0613647904  Library Binding    []   15900
4851   9780785759379  0785759379  Library Binding    []   17511
4930   9785040019526  5040019521        Paperback    []   17837
5256   9780822003908  0822003902        Paperback    []   18984
5470   9780563388692  0563388692         Audio CD    []   19786
5472   9780563388326  0563388323         Audio CD    []   19788
6220   9780613046886  0613046889  Library Binding    []   23479
6387   9780613589451  0613589459  Library Binding    []   24060
6388   9780613674836  0613674839  Library Binding    []   24062
6392   9780613584661  061358466X  Library Binding    []   24066
7038   9780679451211  0679451218       Flexibound    []   26805
7256   9780439959049  0439959047        Paperback    []   27877
7367   9780881031904  0881031909  Library Binding    []   28396
8502   9788401474729  8401474728        Paperback    []   32679
8916   9780613572170  0613572173  Library Binding    []   34530
9277   9781932051292  1932051295        Paperback    []   36461
9586   9780756615697  0756615690       Flexibound    []   38238
10665  9781416500506  1416500502        Hardcover    []   43491

### **5) REMOVE EXTRANEOUS INFORMATION IN THE FORMAT COLUMN AND GROUP SIMILAR FORMATS**

**5a) Partition the format column into first and second parts**

In [20]:
#Examine the dataframe to correct
dfGR_Scraped_Primary

isbn13        isbn                            format  \
0      9780439785969  0439785960              652 pages, Paperback   
1      9780439655484  043965548X  435 pages, Mass Market Paperback   
2      9780439358071  0439358078              870 pages, Paperback   
3      9780976540601  0976540606              152 pages, Paperback   
4      9780439682589  0439682584             2690 pages, Paperback   
...              ...         ...                               ...   
11122  9781563841552  156384155X              342 pages, Paperback   
11123  9780688093389  0688093388               32 pages, Hardcover   
11124  9780465083619  0465083617              464 pages, Paperback   
11125  9780070183179  0070183171              275 pages, Hardcover   
11126  9780310243564  0310243564              352 pages, Paperback   

                                                   genre  bookID  
0      ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       1  
1      ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...       5  
2      ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       2  
3                                                     []       9  
4      ['Fantasy', 'Young Adult', 'Fiction', 'Magic',...       8  
...                                                  ...     ...  
11122  ['Fiction', 'Survival', 'Post Apocalyptic', 'D...   22128  
11123  ['Picture Books', 'Childrens', 'Animals', 'Fic...    5402  
11124  ['Nonfiction', 'Race', 'Education', 'Social Ju...   16280  
11125                        ['Nonfiction', 'Biography']   40146  
11126                                       ['Theology']   44439  

[11127 rows x 5 columns]

In [21]:
#A function that takes a dataframe column containing text
#then it cleans the text by removing excess spaces
#and splits into two columns: a first part, and the rest
#it also adds a third column which shows the last word, which is not removed from "the rest"

#dataframetoPartition = a dataframe containing a text column
#colnumtoPartition = the column number of the text column to process
#splitChar =  the character on which to make the split (space, slash, semi-colon, etc.)
#numWordinFirstPart = the number of words to put in the first part of the split, DEFAULT = 1
#firstwords = if any word in this array is the first word (before split), then the algorithm take the second word as well
#noSingleLetFirst = if toggled to True, it prevents a single from being put in the first part. Two words are put in this case. DEFAULT = False
#includeLastWord = true/false toggle to make an extra column in the result that shows the last word of the text. DEFAULT = False

def makePartition(dataframetoPartition, colnumtoPartition: int, splitChar: str, colnum_of_Unique_ID: int, firstwords: list, numWordinFirstPart = 1, noSingleLetFirst = False, includeLastWord = False):
    if type(dataframetoPartition) != pd.DataFrame:
        print("ERROR: A dataframe is required as an input but a", dataframetoPartition.__class__.__name__, "was supplied")
        return
    if type(colnumtoPartition) != int:
        print("ERROR: The column to partition must be supplied as an integer but", colnumtoPartition.__class__.__name__, "was supplied")
        return
    if type(splitChar) != str:
        print("ERROR: The character on which to split is not the correct format,", splitChar.__class__.__name__, "was supplied")
        return
    if type(colnum_of_Unique_ID) != int:
        print("ERROR: The column of the unique row identifiers must be supplied as an integer but", colnum_of_Unique_ID.__class__.__name__, "was supplied")
        return
    if type(firstwords) != list and type(firstwords) != set:
        print("The first words specified as exceptions must be supplied as a list or set but", firstwords.__class__.__name__, "was supplied")
        return
    
    array1 = []
    for colitem in dataframetoPartition.iloc[:,colnumtoPartition]:
        #put the firstwords in "set" format to avoid duplicates
        firstwords = set(firstwords)
        #Split the text in each row
        colitem = colitem.split(splitChar)

        #If the first word is one of the exceptions, then include the second word in the result of the first part
        #Only applicable in the case where numWordinFirstPart is 1
        if colitem[0] in firstwords and numWordinFirstPart == 1:
            numFirst = 2        
        #If we don't want ONLY a single letter in the First Part, then we ensure that we have 2 words in the first part... if 2 exist
        #Only applicable in the case where numWordinFirstPart is 1
        elif noSingleLetFirst and numWordinFirstPart == 1 and (len(colitem[0])<=1) and (len(colitem)>1):
            numFirst = 2
        #For anything else, only the first word is put in the First Part
        else:
            if len(colitem) >= numWordinFirstPart:
                numFirst = numWordinFirstPart
            else:
                numFirst = len(colitem)
        #Item1 = first part
        item1 = " ".join(colitem[:numFirst])
        #Item2 = the rest, with spaces between the words
        item2 = " ".join(colitem[numFirst:])

        #If no item2, then move item1 to the item2 position
        if item2 == "":
            item2 = item1
            item1 = ""
        
        #Store each pair in an array
        array1.append([item1, item2])
        fullDF = pd.DataFrame(array1, columns = ["First Part","The rest"])
        
    #Add the full string to the dataframe for comparison
    fullDF["Full String"] = dataframetoPartition.iloc[:,colnumtoPartition]
    #Add the column from the original dataframe that containes the unique identifiers for each line (bookID, ISBN, etc.)
    fullDF[dataframetoPartition.columns[colnum_of_Unique_ID]] = dataframetoPartition.iloc[:, colnum_of_Unique_ID]
    #Add a column showing the last word in the same array
    if includeLastWord:
        fullDF["Last Word Alone"] = dataframetoPartition.iloc[:,colnumtoPartition].str.rpartition(sep=" ", expand = True).drop([1], axis=1)[2]
    return fullDF

In [22]:
#Use the makePartition function on the format column
splitFormat = makePartition(dfGR_Scraped_Primary, colnumtoPartition = 2, splitChar = ",", numWordinFirstPart = 1, colnum_of_Unique_ID = 4, firstwords = [""], includeLastWord = True)
splitFormat

First Part                The rest                       Full String  \
0       652 pages               Paperback              652 pages, Paperback   
1       435 pages   Mass Market Paperback  435 pages, Mass Market Paperback   
2       870 pages               Paperback              870 pages, Paperback   
3       152 pages               Paperback              152 pages, Paperback   
4      2690 pages               Paperback             2690 pages, Paperback   
...           ...                     ...                               ...   
11122   342 pages               Paperback              342 pages, Paperback   
11123    32 pages               Hardcover               32 pages, Hardcover   
11124   464 pages               Paperback              464 pages, Paperback   
11125   275 pages               Hardcover              275 pages, Hardcover   
11126   352 pages               Paperback              352 pages, Paperback   

       bookID Last Word Alone  
0           1       Paperback  
1           5       Paperback  
2           2       Paperback  
3           9       Paperback  
4           8       Paperback  
...       ...             ...  
11122   22128       Paperback  
11123    5402       Hardcover  
11124   16280       Paperback  
11125   40146       Hardcover  
11126   44439       Paperback  

[11127 rows x 5 columns]

**5b) Convert the column of interest "The rest" into lowercase and remove extra spaces**

In [23]:
#Make second column "The rest" lowercase
splitFormat.iloc[:,1] = splitFormat.iloc[:,1].str.lower()

In [24]:
#Function to verify number of characters
#Used in the removeExtraSpaces function
def charCounter(listToVerify: str, char: str):
    count = 0
    for i in listToVerify:
        count += i.count(char)
    return count
    
#Function to remove extra spaces (at beginning, end, or in the middle)
#Count double spaces before and after cleaning
def removeExtraSpaces(inputList: str):
    before = charCounter(inputList, char = "  ")
    correctedList = []
    for item in inputList:
        #Split the text in each row
        item = item.split(" ")

        #Remove the spaces, since there are sometimes excess spaces
        nbocc = item.count('')
        [item.remove('') for i in range(nbocc)]

        item = " ".join(item[:])
        correctedList.append([item])
    after = charCounter(correctedList, char = "  ")
    print("Double spaces before =", before,"\tDoublespaces after =", after)
    return correctedList

In [25]:
#Clean all extra spaces from the title column:
colName = "The rest"
splitFormat[colName] = pd.DataFrame(removeExtraSpaces(splitFormat[colName]))

Double spaces before = 0 	Doublespaces after = 0


**5c) Examine the number of occurences of each format and group similar formats together**

In [26]:
#Count the occurences of each format
splitFormat["The rest"].value_counts()

The rest
paperback                7242
hardcover                2076
mass market paperback    1355
audio cd                  203
library binding            50
audio cassette             30
paper                      20
board book                 15
pocket book                11
ebook                      10
unknown binding            10
cloth                       9
audiobook                   8
audio                       8
コミック                        7
trade paperback             6
mp3 cd                      6
perfect paperback           6
spiral-bound                5
comic                       5
capa mole                   5
hardback                    5
leather bound               5
kindle edition              4
taschenbuch                 3
cd-rom                      2
softcover                   2
product bundle              2
flexibound                  2
dvd and book                1
bath book                   1
320 pages                   1
staple bound                1
p

In [27]:
#Make a dictionary to reclassify certain formats (grouping together formats that should be the same)
replacement_dictionary = {
    "mass market paperback": "paperback",
    "audio cd": "audio",
    "audio cassette": "audio",
    "paper": "paperback",
    "unknown binding": "unknown",
    "audiobook": "audio",
    "コミック": "comic",
    "trade paperback": "paperback",
    "mp3 cd": "audio",
    "perfect paperback": "paperback",
    "capa mole": "paperback",
    "hardback": "hardcover",
    "kindle edition": "ebook",
    "taschenbuch": "hardcover",
    "cd-rom": "audio",
    "dvd and book": "product bundle",
    "320 pages": "unknown",
    "paperback manga": "comic",
    "comics": "comic",
    "58 pages": "unknown",
    "poche": "paperback",
    "slipcased hardcover": "hardcover",
    "textbook binding": "library binding",
    "publisher's binding": "unknown",
    "240 pages": "unknown"
}

In [28]:
#Replace formats using the dictionary
splitFormat["The rest"].replace(replacement_dictionary, inplace=True)
splitFormat["The rest"].value_counts()

The rest
paperback            8635
hardcover            2085
audio                 257
library binding        51
unknown                15
board book             15
comic                  14
ebook                  14
pocket book            11
cloth                   9
leather bound           5
spiral-bound            5
product bundle          3
softcover               2
flexibound              2
imitation leather       1
unbound                 1
staple bound            1
bath book               1
Name: count, dtype: int64

**5d) Place the corrected formats into the primary dataframe**

In [29]:
#Add the corrected formats to the primary dataframe (adjusting the column labels as necessary)
dfGR_Scraped_Primary = dfGR_Scraped_Primary.merge(splitFormat[["bookID", "The rest"]], how="inner", on="bookID")
dfGR_Scraped_Primary.rename(mapper={"format": "scraped_format"}, axis=1, inplace=True)
dfGR_Scraped_Primary.rename(mapper={"The rest": "format"}, axis=1, inplace=True)

### **6) REFORMAT THE GENRE COLUMN TO MAKE IT EXPLOITABLE**

**6a) Correct the data in the genre column (make as a list and lowercase)**

In [30]:
#Convert the genre to lower case
dfGR_Scraped_Primary["genre"] = dfGR_Scraped_Primary["genre"].str.lower()

In [31]:
#Convert the items in the genre column to real lists (without the apostrophes)
dfGR_Scraped_Primary.genre = dfGR_Scraped_Primary.genre.apply(eval)

In [32]:
#Explode the genre column into serveral columns
genres_lowercase = []
for index, genre_item in enumerate(dfGR_Scraped_Primary["genre"]):
    genres_lowercase.append([item.lower() for item in genre_item])
dfGR_Scraped_Primary = pd.concat([dfGR_Scraped_Primary, pd.DataFrame(genres_lowercase)], axis=1)

**6b) Create a reduced dataframe and export to file**

In [33]:
dfGR_Scraped_Primary.drop(labels=['isbn', 'isbn13', 'scraped_format'], axis=1, inplace=True)
dfGR_Scraped_Primary

genre  bookID     format  \
0      [fantasy, young adult, fiction, magic, childre...       1  paperback   
1      [fantasy, fiction, young adult, magic, childre...       5  paperback   
2      [fantasy, young adult, fiction, magic, childre...       2  paperback   
3                                                     []       9  paperback   
4      [fantasy, young adult, fiction, magic, adventu...       8  paperback   
...                                                  ...     ...        ...   
11122  [fiction, survival, post apocalyptic, dystopia...   22128  paperback   
11123  [picture books, childrens, animals, fiction, h...    5402  hardcover   
11124  [nonfiction, race, education, social justice, ...   16280  paperback   
11125                            [nonfiction, biography]   40146  hardcover   
11126                                         [theology]   44439  paperback   

                   0            1                 2               3  \
0            fantasy  young adult           fiction           magic   
1            fantasy      fiction       young adult           magic   
2            fantasy  young adult           fiction           magic   
3               None         None              None            None   
4            fantasy  young adult           fiction           magic   
...              ...          ...               ...             ...   
11122        fiction     survival  post apocalyptic        dystopia   
11123  picture books    childrens           animals         fiction   
11124     nonfiction         race         education  social justice   
11125     nonfiction    biography              None            None   
11126       theology         None              None            None   

                 4             5                6  
0        childrens     audiobook        adventure  
1        childrens  middle grade        audiobook  
2        childrens     audiobook        adventure  
3             None          None             None  
4        adventure     childrens     supernatural  
...            ...           ...              ...  
11122  apocalyptic      politics  science fiction  
11123        humor     storytime           africa  
11124  anti racist    psychology        sociology  
11125         None          None             None  
11126         None          None             None  

[11127 rows x 10 columns]

In [34]:
#Export the dataframe to a tsv file
dfGR_Scraped_Primary.to_csv(repPath + 'scraped_data_genres_formats_cleaned.tsv', sep='\t')